# Collision Avoidance - Train Model (衝突回避 - ResNet18モデルの学習)

このノートブックでは、衝突回避のために``free「直進する」``と``blocked「旋回する」``の2つのクラスを特定する画像分類モデルを学習します。  
モデルの学習には人気のあるディープラーニングライブラリの *PyTorch* を使います。

PyTorchのResNet18モデル構造はTensorRTへの変換がサポートされているため、学習したモデルは簡単にTensorRTモデルに変換することが可能です。  
TensorRTモデルへの変換を行うと、推論の実行速度およびメモリ使用量を最大限に効率化することが可能になります。

## それでは、始めましょう
まずは必要なライブラリを読み込みます。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

### データセットインスタンスを作成

### データセットインスタンスを生成

[torchvision.datasets](https://pytorch.org/docs/stable/torchvision/datasets.html)パッケージに含まれる``ImageFolder`` classを使用します。学習用のデータを準備するために、``torchvision.transforms``パッケージを使って画像変換を定義します。  
pytorch ImageNetの学習に使われた140万件のデータセットから得られる各チャンネルの平均と標準偏差は以下の値になります。  
RGB各要素の平均：[0.485, 0.456, 0.406]  
RGB各要素の標準偏差：[0.229, 0.224, 0.225]  
正規化に関する部分はライブデモの「カメラ画像の前処理作成」の部分でもう少し詳しく説明してあります。

* ImageFolderリファレンス：
  * https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder
* ImageFolder実装コード：
  * https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py
* Normalizeリファレンス：
  * https://pytorch.org/docs/stable/torchvision/transforms.html
* Normalize実装コード：
  * https://github.com/pytorch/vision/blob/master/torchvision/transforms/transforms.py
* 正規化パラメータの値の理由：
  * https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2
* 正規化に意味があるのかどうか：
  * https://teratail.com/questions/234027

In [ ]:
########################################
# jpeg画像データを学習可能なデータフォーマットに変換して提供するインスタンスを作成します。
# このデータセットは、アクセス毎にtransformsを実行するため、ColorJitterにより毎回色合いがランダムに変化します。
# dataset変数には以下のような値が入ります。
# dataset[データ番号][0]：(3x224x224)のCHWデータ（ColorJitterによりアクセス毎に色合いがランダムに変化します）
# dataset[データ番号][1]：ラベル番号（blockedの場合は0、freeの場合は1）
########################################
dataset = datasets.ImageFolder(
    'dataset',  # データセットのディレクトリパスを指定します。
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),  # カラージッターは画像の明るさ、コントラスト、彩度をランダムに変更します。
        transforms.Resize((224, 224)),  # 画像サイズを224x224にリサイズします。
        transforms.ToTensor(),  # cudnnはHWC(Height x Width x Channel)をサポートしません。そのため画像(HWC layout)からTensor(CHW layout)に変換します。また、RGB各値を[0, 255]から[0.0, 1.0]の範囲にスケーリングします。
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # この値はpytorch ImageNetの学習に使われた正規化（ImageNetデータセットのRGB毎に平均を0、標準偏差が1になるようにスケーリングすること）のパラメータです。学習済みモデルをベースとした転移学習を行うため、カメラ画像はこの値でRGBを正規化することが望ましいでしょう。
    ])
)

### トレーニングデータとテストデータに分ける
次に、データセットを*トレーニング用*と*テスト用*のデータセットに分割します。この例では、*トレーニング用*に50%, *テスト用*に50%で分けます。  
*テスト用*のデータセットは、学習中にモデルの精度を検証するために使用されます。

In [ ]:
########################################
# 学習に使うデータと学習させずに精度検証のために使うデータに分けます。
# blocked、freeの順で読込まれていたデータをランダムに抽出して、学習用と検証用にデータを分けます。
########################################
test_percent = 0.5 # テストデータ件数を50%にする
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### バッチ処理で学習データとテストデータを読み込むためのデータローダーを作成

[torch.utils.data.DataLoader](https://github.com/pytorch/pytorch/blob/master/torch/utils/data/dataloader.py)クラスは、モデル学習中に次のデータ処理が完了出来るようにサブプロセスで並列処理にして実装します。  
データのシャッフル、バッチでのデータロードのために使用します。この例では、1回のバッチ処理で8枚の画像を使用します。これをバッチサイズと呼び、GPUのメモリ使用量と、モデルの精度に影響を与えます。

* DataLoaderリファレンス：
  * https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
* DataLoader実装コード：
  * https://github.com/pytorch/pytorch/blob/master/torch/utils/data/dataloader.py

In [ ]:
########################################
# データセットを分割読込みするためのデータローダーを作成します。
########################################
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)

### JetBot用にモデルを変更する

torchvisionで使用可能なImageNetデータセットで学習済みのResNet18モデルを使用します。

*転移学習*と呼ばれる手法で、すでに画像分類できる特徴を持つニューラルネットワーク層を、別の目的のために作られたモデルに適用することで、短時間で良好な結果を得られるモデルを作成することができます。

* ResNet18の詳細
  * https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
* 転移学習の詳細
  * https://www.youtube.com/watch?v=yofjFQddwHE

In [ ]:
########################################
# PyTorchで提供されているImageNetデータセットで学習済みのResNet18モデルを読込みます。
########################################
model = models.resnet18(pretrained=True)
# モデルを凍結して使う場合は、全てのパラメータが持つ学習フラグを無効化します。デフォルトはrequires_grad = Trueで全てのパラメータを再学習します。
#for param in model.parameters():
#    param.requires_grad = False

ResNet18モデルはImageNetを学習するために作られているため、1000種類の画像分類が可能な出力を持っています。しかしJetBotの衝突回避モデルの出力は「free」と「blocked」の2種類しか用意していません。  
`ResNet18`モデルは出力層の手前にある層が512のノード数を持っているので、ResNet18モデル構造の全結合層(fully connected layer)を入れ替えて、モデルの出力層を(512,2)に置き換えて使います。

In [ ]:
########################################
# モデルの出力層をJetBotの衝突回避モデル用に置き換えます。
########################################
print(model.fc.in_features)
model.fc = torch.nn.Linear(model.fc.in_features, 2)

デフォルトではモデルのweightの計算はCPUで処理されるため、GPUを利用するようにモデルを設定します。

In [ ]:
########################################
# GPU処理が可能な部分をGPUで処理するように設定します。
########################################
device = torch.device('cuda')
model = model.to(device)

### モデルの学習
30エポック学習し、各エポックでテストデータにおけるこれまでの最高精度と現在の精度を比較することにより、最高精度を更新した場合に保存します。  
最高精度が同じ場合は損失を更新した場合に保存します。

> 1エポックは、私たちが用意したトレーニング用のデータ全部を1回学習することです。データローダーのbatch_sizeで指定した数の画像を一度に学習するミニバッチ処理を複数回実行することで1エポックが完了します。

* バックプロパゲーションと勾配更新の関係について：
  * https://stackoverflow.com/questions/53975717/pytorch-connection-between-loss-backward-and-optimizer-step
* torch.tensorとnp.ndarrayの違いについて：
  * https://stackoverflow.com/questions/63582590/why-do-we-call-detach-before-calling-numpy-on-a-pytorch-tensor/63869655#63869655

In [ ]:
NUM_EPOCHS = 30  # 学習するエポック数。
BEST_MODEL_PATH = 'best_model_resnet18.pth'  # 学習結果を保存するファイル名。
best_test_accuracy = 0.0  # 検証用の初期精度は0%の精度を意味する0.0としておきます。
saved_loss = 1e9  # 保存時の評価損失は10の9乗(1000000000)を意味する1e9としておきます。

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # 確率的勾配降下法を実装します。学習率と運動量はJetBotの学習に適した値を指定します。

########################################
# 学習を開始します。
########################################
print("学習を開始します。")
for epoch in range(NUM_EPOCHS):

    ####################
    # SGDに基づいた学習をバッチ毎に実行します。
    ####################
    train_loss = 0.0  # 損失を0.0として初期化します。
    train_error_count = 0.0  # 不一致件数を0件として初期化します。
    model.train()  # モデルを学習モードに設定します。
    for images, labels in iter(train_loader):  # batch_size分の学習データを読み込みます。
        images = images.to(device)  # 画像データをGPUメモリに転送します。
        labels = labels.to(device)  # ラベルデータをGPUメモリに転送します。
        optimizer.zero_grad()  # 最適化されたすべてのtorch.Tensorの勾配をゼロに設定します。
        outputs = model(images)  # batch_size分の予測を一度に実行します。
        loss = F.cross_entropy(outputs, labels)  # batch_size分のモデルの予測結果と正解ラベルを照合して損失を計算します。
        train_loss += loss # 1epochの損失を求めるために、今回のバッチの損失を加算します。
        train_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1)))) # for graph
        loss.backward()  # 各パラメータ毎の損失の勾配を計算します。
        optimizer.step()  # 計算された損失勾配で各パラメータの勾配を更新します。
    train_loss /= len(train_loader)  # 1epochの損失からデータ1件あたりの学習損失を求めます。

    ####################
    # 未学習のデータでモデルの精度を検証します。
    ####################
    test_loss = 0.0  # 損失を0.0として初期化します。
    test_error_count = 0.0  # 不一致件数を0件として初期化します。
    model.eval()  # モデルをテストモードに設定します。
    for images, labels in iter(test_loader):  # batch_size分の学習データを読み込みます。
        images = images.to(device)  # 画像データをGPUメモリに転送します。
        labels = labels.to(device)  # ラベルデータをGPUメモリに転送します。
        outputs = model(images)  # バッチ1回分のモデルの予測結果と正解ラベルを照合して損失を計算します。
        test_loss += loss  # 1epochの損失を求めるために、今回のバッチの損失を加算します。
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))  # 予測結果と正解ラベルが不一致となった件数をカウントします。
    test_loss /= len(test_loader)  # 1epochの損失からデータ1件あたりの評価損失を求めます。

    ####################
    # 学習結果をログに表示します。
    ####################
    train_accuracy = 1.0 - float(train_error_count) / float(len(train_dataset))  # 不一致となった割合から精度を算出します。
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))  # 不一致となった割合から精度を算出します。

    ####################
    # 今回のepoch学習のテスト結果がよければ保存します
    ####################
    is_saved = False
    if test_accuracy > best_test_accuracy:  # 過去最高の精度を更新した場合は保存します。
        torch.save(model.state_dict(), BEST_MODEL_PATH)  # 学習したモデルをファイルに保存します。
        best_test_accuracy = test_accuracy  # 過去最高の精度として値を保持します。
        saved_loss = test_loss  # 保存時の評価損失として値を保持します。
        is_saved = True  # 保存フラグを更新します。
    elif test_accuracy == best_test_accuracy:  # 精度が同じ場合は評価損失でモデルを保存するか判断します。
        if test_loss < saved_loss:  # 評価損失を更新した場合は保存します。
            torch.save(model.state_dict(), BEST_MODEL_PATH)  # 学習したモデルをファイルに保存します。
            saved_loss = test_loss  # 保存時の評価損失として値を保持します。
            is_saved = True  # 保存フラグを更新します。

    # 学習結果をログに表示します。
    print('%d: %f, %f, %f, %f, ' % (epoch+1, train_loss, test_loss, train_accuracy, test_accuracy)+("saved" if is_saved else "not saved"))

print("学習が終了しました。")

学習が完了すると、``03_convert_to_trt.ipynb``で推論に使う``best_model_resnet18.pth``が生成されます。

## 次
JetBot本体で学習した場合は、このノートブックを閉じてからJupyter左側にある「Running Terminals and Kernels」を選択して「02_train_model_resnet18.ipynb」の横にある「SHUT DOWN」をクリックしてJupyter Kernelをシャットダウンしてから[03_convert_to_trt.ipynb](03_convert_to_trt.ipynb)に進んでください。  

TensroRTへの変換をおこなわずに動作確認したい場合は[05_live_demo_resnet18.ipynb](05_live_demo_resnet18.ipynb)に進んでください。